In [1]:
!pip install praw transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 139.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 124.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [12]:
import praw

CLIENT_ID = "RF6R8bUh5oIKByB6QJvUbw"
CLIENT_SECRET = "Ag2Li4SIzrU2sA8kEkBnJsk2qfZYYQ"
USER_AGENT = "script:RedditPersonaApp:1.0 (by u/Wide-Poetry-1575)"

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

In [13]:
import re
from transformers import pipeline

def extract_username(url_or_username: str) -> str:
    m = re.search(r"reddit\\.com/user/([^/]+)/?", url_or_username)
    return m.group(1) if m else url_or_username

def get_user_data(username: str, limit: int = 100):
    user = reddit.redditor(username)
    return (
        [f"{sub.title}: {sub.selftext}" for sub in user.submissions.new(limit=5)],
        [c.body for c in user.comments.new(limit=10)]
    )

def build_prompt(posts, comments):
    intro = """You are a persona expert. Here's an example persona format:

1. Personality traits: Curious, introverted, honest.
2. Interests and hobbies: Loves comic books, interested in AI and psychology.
3. Communication style: Uses humor and sarcasm; prefers short replies.
4. Beliefs and opinions: Supports mental health awareness.
5. Behavioral patterns: Active in niche subreddits; replies quickly.

Now, analyze the user based on the following Reddit content:

"""

    text = ""
    for post in posts[:2]:
        text += f"- Post: {post[:150]}\n"
    for comment in comments[:3]:
        text += f"- Comment: {comment[:150]}\n"

    return intro + text

generator = pipeline("text2text-generation", model="google/flan-t5-xl")

def generate_persona_text(prompt: str) -> str:
    out = generator(prompt, max_new_tokens=500, do_sample=True,
                    temperature=0.7, top_k=50, top_p=0.95,
                    repetition_penalty=1.2)
    return out[0].get("generated_text", "").strip()

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [18]:
user_input = "Hungry-Move-6603"  # Change to any Reddit username
username = extract_username(user_input)
posts, comments = get_user_data(username)
prompt = build_prompt(posts, comments)

# 🔥 Generate the persona
persona = generate_persona_text(prompt)

# ✅ Display final output
print(f"\n=== Reddit Persona for {username} ===\n\n{persona}")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




=== Reddit Persona for Hungry-Move-6603 ===

[removed] is an introverted, honest user who is active in niche subreddits. He prefers humor and sarcasm; prefers short replies. He supports mental health awareness.


In [19]:
import os
os.makedirs("personas", exist_ok=True)

with open(f"personas/{username}_persona.txt", "w", encoding="utf-8") as f:
    f.write(persona)

print(f"Saved to personas/{username}_persona.txt")


Saved to personas/Hungry-Move-6603_persona.txt
